In [1]:

import ipywidgets, datetime, pandas, random
import requests, pydash, altair
from IPython.display import clear_output

country_one = ipywidgets.Dropdown()
country_two = ipywidgets.Dropdown()
button = ipywidgets.Button(description="run report")

output = ipywidgets.Output()
display(country_one, country_two, button, output)

url = 'http://138.197.181.117:8989/bigdata/sparql'

query = """
SELECT DISTINCT ?country ?countryLabel
WHERE
{
  ?person wdt:P5 wd:Q2 . # people
  ?person wdt:P9 ?country

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

data = pydash.get(data, 'results.bindings')
data = [(pydash.get(x, 'countryLabel.value'),pydash.get(x, 'country.value')) for x in data]
data = {x: y.split('/')[-1] for x,y in data}
countries = sorted([x for x in data.keys()])

country_one.options = countries
country_one.value = random.choice(countries)
country_two.options = countries
country_two.value = random.choice(countries)

def on_button_clicked(b):
    with output:
        if country_one.value != None and country_two.value != None:
 
            clear_output()
            print('🤔')
            data_concat = pandas.DataFrame()

            for x in [country_one.value, country_two.value]:

                country_code = data[x]

                query = """

                SELECT ?year 
                WHERE
                {
                  ?person wdt:P5 wd:Q2 . # people
                  ?person wdt:P9 wd:"""+country_code+""" . # people of country x
                  ?film wdt:P6 ?person . # films voted by people
                  ?film wdt:P1 ?year . # year of the film voted by people from the country

                  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
                }"""
                r = requests.get(url, params = {'format': 'json', 'query': query})
                data_string = r.json()

                data_string = pydash.get(data_string, 'results.bindings')              
                data_string = [pydash.get(x, 'year.value') for x in data_string]
                data_string = [x.split('.')[0] for x in data_string]             
                data_string = [x for x in data_string if x in [str(x) for x in range(1880,2020)]]
                index = pandas.Index(data_string)
                collapsed = pandas.DataFrame(index.value_counts()).reset_index()
                collapsed.columns = ['year', 'votes']

                years = pandas.DataFrame([str(x) for x in range(1880,2012)], columns=['year'])
                years = pandas.merge(years, collapsed, on='year', how='left')
                years = years.fillna('0')
                years['country'] = x

                data_concat = pandas.concat([data_concat, years])

            clear_output()
            line = altair.Chart(data_concat).mark_line(interpolate='basis').encode(
                                x='year:T',y='votes:Q',color='country:N')
            display(altair.layer(line).properties(width=600, height=300))
        else:
            print('please select a valid country.')
            
button.on_click(on_button_clicked)


Dropdown(options=(), value=None)

Dropdown(options=(), value=None)

Button(description='run report', style=ButtonStyle())

Output()